### Analysis of data reported from HPC grid search of hyperparameters

Tasks:
    1. obtain table for the master report for drug-by-drug learning
    2. figure out the  best models for each coefficient
    3. save the best hyperparameters for best models for each coefficients

In [1]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

_FOLDER = "GridSearch_results/"
_FOLDER_2 = "results/"

In [2]:
model_names =['lasso',
        'ridge',
    'Linear_KernelRidge',
    'Sigmoid_KernelRidge',
    'RBF_KernelRidge',
    'PolynomialKernelRidge',
    'Linear_SVR',
    'Sigmoid_SVR',
    'RBF_SVR',
    "Polynomial_SVR"]

### 1. Table for report

In [3]:
df_results_mae = pd.DataFrame(index=model_names, columns =["coef_"+str(i) for i in range(1,5)])

for model in model_names:
    for i in range(1,5):
        df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv")
        if i!=3 :
            df_results_mae.loc[model, "coef_"+str(i)] = str(round(df["mae_3"].mean(), 3)) + " / " + str(round(df["mae_3"].std(),3))
        else:
            df_results_mae.loc[model, "coef_"+str(i)] = str(round(df["mae_3"].mean(), 2)) + " / " + str(round(df["mae_2"].std(),2))

df_results_mae.to_csv("results_drug_by_drug.csv")

### 2. Best models for the coefficients

In [4]:
df_results = pd.DataFrame(index=model_names, columns =["coef_"+str(i) for i in range(1,5)])
for model in model_names:
    for i in range(1,5):
        df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv")
        df_results.loc[model, "coef_"+str(i)] = df["mae_3"].mean()
df_results

,coef_1,coef_2,coef_3,coef_4
lasso,0.341283,0.391105,11.1317,0.0779428
ridge,0.374303,0.410487,10.8322,0.0821313
Linear_KernelRidge,0.358402,0.677494,12.2731,0.0759723
Sigmoid_KernelRidge,0.25027,0.390815,9.84057,0.0787291
RBF_KernelRidge,0.245373,0.377003,9.68135,0.0746973
PolynomialKernelRidge,0.236148,0.376241,9.75768,0.0758253
Linear_SVR,0.241344,0.280438,9.42036,0.0846662
Sigmoid_SVR,0.241714,0.275298,9.50708,0.0840374
RBF_SVR,0.237239,0.271848,9.50916,0.0754618
Polynomial_SVR,0.239229,0.273368,9.58911,0.0782375


In [5]:
best_models = {}
for i in range(1,5):
    best_models["coef_" + str(i)] = list(df_results[df_results["coef_"+str(i)] == df_results["coef_"+str(i)].min()].index)
    print("coef_" + str(i) +":", list(df_results[df_results["coef_"+str(i)] == df_results["coef_"+str(i)].min()].index))

coef_1: ['PolynomialKernelRidge']
coef_2: ['RBF_SVR']
coef_3: ['Linear_SVR']
coef_4: ['RBF_KernelRidge']


### 3. Hyperparameters for the best model

In [6]:
best_models

{'coef_1': ['PolynomialKernelRidge'],
 'coef_2': ['RBF_SVR'],
 'coef_3': ['Linear_SVR'],
 'coef_4': ['RBF_KernelRidge']}

In [7]:
parameters_to_use = {
    'lasso' : ["best_alpha3"],
    'ridge' : ["best_alpha3", "best_solver3"],
    'Linear_KernelRidge' : ["best_alpha3"],
    'Sigmoid_KernelRidge': ['best_alpha3', 'best_gamma3', 'best_coef03'],
    'RBF_KernelRidge': ['best_alpha3', 'best_gamma3', 'best_coef03'],
    'PolynomialKernelRidge' : ['best_alpha3', 'best_gamma3', 'best_coef03', 'best_degree3'],
    'Linear_SVR' : ['best_C3', 'best_epsilon3'],
    'Sigmoid_SVR' : ['best_C3', 'best_epsilon3', 'best_coef03'],
    'RBF_SVR':['best_C3', 'best_epsilon3', 'best_coef03'],
    "Polynomial_SVR": ['best_C3', 'best_epsilon3', 'best_coef03', 'best_degree3']
}

In [8]:
i=1
model = best_models["coef_"+str(i)][0] 
print("Coefficient", i, ":", model)
df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"})
best = df[['DRUG_ID']+parameters_to_use[model]]
best.to_csv(_FOLDER_2 +"best_model_coef_" +str(i)+ ".csv")
best

Coefficient 1 : PolynomialKernelRidge


,DRUG_ID,best_alpha3,best_gamma3,best_coef03,best_degree3
0,170,100.0,0.10000,5.0,1.0
1,173,100.0,0.01000,5.0,3.0
2,180,100.0,0.00001,0.5,1.0
3,200,100.0,0.00001,5.0,1.0
4,219,10.0,0.00001,0.5,2.0
5,272,500.0,0.00010,5.0,2.0
6,273,0.5,0.01000,1.0,1.0
7,274,1.0,0.00001,0.5,5.0
8,276,1.0,0.00001,5.0,5.0
9,328,5.0,0.00010,0.5,3.0


In [9]:
i=2
model = best_models["coef_"+str(i)][0] 
print("Coefficient", i, ":", model)
df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"})
best = df[['DRUG_ID']+parameters_to_use[model]]
best.to_csv(_FOLDER_2+"best_model_coef_" +str(i)+ ".csv")
best

Coefficient 2 : RBF_SVR


,DRUG_ID,best_C3,best_epsilon3,best_coef03
0,170,0.1,0.010,0.01
1,173,0.1,0.001,0.01
2,180,0.1,0.010,0.01
3,200,0.1,0.010,0.01
4,219,0.5,0.010,0.01
5,272,0.1,0.001,0.01
6,273,0.1,0.001,0.01
7,274,0.1,0.010,0.01
8,276,0.1,0.010,0.01
9,328,0.1,0.001,0.01


In [10]:
i=3
model = best_models["coef_"+str(i)][0] 
print("Coefficient", i, ":", model)
df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"})
best = df[['DRUG_ID']+parameters_to_use[model]]
best.to_csv(_FOLDER_2+"best_model_coef_" +str(i)+ ".csv")
best

Coefficient 3 : Linear_SVR


,DRUG_ID,best_C3,best_epsilon3
0,170,0.10,5.000
1,173,0.10,0.001
2,180,5.00,5.000
3,200,0.01,0.100
4,219,0.10,1.000
5,272,0.01,2.000
6,273,0.01,2.000
7,274,0.10,0.100
8,276,0.01,1.000
9,328,0.01,2.000


In [11]:
i=4
model = best_models["coef_"+str(i)][0] 
print("Coefficient", i, ":", model)
df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"})
best = df[['DRUG_ID']+parameters_to_use[model]]
best.to_csv(_FOLDER_2+"best_model_coef_" +str(i)+ ".csv")
best

Coefficient 4 : RBF_KernelRidge


,DRUG_ID,best_alpha3,best_gamma3,best_coef03
0,170,50.0,0.00001,0.01
1,173,0.1,1.00000,0.01
2,180,10.0,0.00010,0.01
3,200,0.1,0.00010,0.01
4,219,5.0,0.00001,0.01
5,272,10.0,0.00001,0.01
6,273,50.0,0.01000,0.01
7,274,100.0,0.00001,0.01
8,276,50.0,0.00001,0.01
9,328,0.1,0.00010,0.01


### Additional : Checking hyperparameters amond datasets

In [12]:
i=1
for model in model_names:
    print(model)
    df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"}) 
    display(df)
    print("MAE among datasets:")
    display(round(df[["mae_"+str(i) for i in range(4)]], 4))
    print("Coefficient", i, ":", model, round(df["mae_3"].mean(), 3), "+/-", round(df["mae_3"].std(),3))
    print("--------------------------------\n")

lasso


,DRUG_ID,best_alpha0,mae_0,mse_0,intercept_0,sum_coef_0,best_alpha1,mae_1,mse_1,intercept_1,...,best_alpha2,mae_2,mse_2,intercept_2,sum_coef_2,best_alpha3,mae_3,mse_3,intercept_3,sum_coef_3
0,170,0.50,0.124879,0.022347,0.580835,0.000000,0.50,0.124879,0.022347,0.580835,...,0.50,0.124879,0.022347,0.580835,0.000000,0.50,0.124879,0.022347,0.580835,0.000000
1,173,0.10,0.135350,0.050504,0.614222,0.000000,0.10,0.135350,0.050504,0.614222,...,0.10,0.135350,0.050504,0.614222,0.000000,0.10,0.135350,0.050504,0.614222,0.000000
2,180,1.00,2.502282,18.146173,1.571673,0.000000,1.00,2.502282,18.146173,1.571673,...,1.00,2.502282,18.146173,1.571673,0.000000,1.00,2.502282,18.146173,1.571673,0.000000
3,200,0.01,0.122048,0.019527,0.472703,8.319665,0.01,0.122048,0.019527,0.472703,...,0.01,0.122048,0.019527,0.472703,8.319665,0.01,0.122048,0.019527,0.472703,8.319665
4,219,0.50,0.192086,0.044297,0.669953,0.000000,0.50,0.192086,0.044297,0.669953,...,0.50,0.192086,0.044297,0.669953,0.000000,0.50,0.192086,0.044297,0.669953,0.000000
5,272,0.50,0.159842,0.034059,0.664511,0.000000,0.50,0.159842,0.034059,0.664511,...,0.50,0.159842,0.034059,0.664511,0.000000,0.50,0.159842,0.034059,0.664511,0.000000
6,273,0.01,0.062783,0.005959,0.588298,0.020031,0.01,0.062783,0.005959,0.588298,...,0.01,0.062783,0.005959,0.588298,0.020031,0.01,0.062783,0.005959,0.588298,0.020031
7,274,0.50,0.184657,0.040335,0.604637,0.000000,0.50,0.184657,0.040335,0.604637,...,0.50,0.184657,0.040335,0.604637,0.000000,0.50,0.184657,0.040335,0.604637,0.000000
8,276,0.10,0.122949,0.019273,0.528040,0.000000,0.10,0.122949,0.019273,0.528040,...,0.10,0.122949,0.019273,0.528040,0.000000,0.10,0.122949,0.019273,0.528040,0.000000
9,328,0.01,0.069049,0.007266,0.349147,11.251302,0.01,0.069049,0.007266,0.349147,...,0.01,0.069049,0.007266,0.349147,11.251302,0.01,0.069049,0.007266,0.349147,11.251302


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.1249,0.1249,0.1249,0.1249
1,0.1354,0.1354,0.1354,0.1354
2,2.5023,2.5023,2.5023,2.5023
3,0.1220,0.1220,0.1220,0.1220
4,0.1921,0.1921,0.1921,0.1921
5,0.1598,0.1598,0.1598,0.1598
6,0.0628,0.0628,0.0628,0.0628
7,0.1847,0.1847,0.1847,0.1847
8,0.1229,0.1229,0.1229,0.1229
9,0.0690,0.0690,0.0690,0.0690


Coefficient 1 : lasso 0.341 +/- 0.718
--------------------------------

ridge


...

Coefficient 1 : RBF_SVR 0.237 +/- 0.483
--------------------------------

Polynomial_SVR


,DRUG_ID,best_C0,best_epsilon0,best_coef00,best_degree0,mae_0,mse_0,best_C1,best_epsilon1,best_coef01,...,best_coef02,best_degree2,mae_2,mse_2,best_C3,best_epsilon3,best_coef03,best_degree3,mae_3,mse_3
0,170,10.0,0.001,0.10,4.0,0.124124,0.026509,10.0,0.001,0.10,...,0.10,4.0,0.124130,0.026514,10.0,0.001,0.10,4.0,0.124195,0.026555
1,173,0.5,0.001,0.01,1.0,0.145646,0.055158,0.5,0.001,0.10,...,0.01,1.0,0.145629,0.055147,0.5,0.001,5.00,1.0,0.145653,0.055188
2,180,0.1,0.001,0.01,1.0,1.690230,20.713848,0.1,0.001,5.00,...,1.00,1.0,1.690200,20.713821,0.1,0.001,0.50,1.0,1.690167,20.714300
3,200,0.1,0.100,0.10,1.0,0.126306,0.020871,0.1,0.100,1.00,...,0.01,1.0,0.126303,0.020870,0.1,0.100,0.01,1.0,0.126289,0.020861
4,219,0.1,0.010,0.10,1.0,0.069566,0.011100,0.1,0.010,5.00,...,5.00,1.0,0.069560,0.011099,0.1,0.010,5.00,1.0,0.069486,0.011096
5,272,0.5,0.010,0.01,2.0,0.081467,0.009968,0.5,0.010,0.01,...,0.01,2.0,0.081438,0.009964,0.5,0.010,0.01,2.0,0.081353,0.009937
6,273,0.1,0.001,0.10,2.0,0.068807,0.007264,0.1,0.001,0.10,...,0.10,2.0,0.068804,0.007264,0.1,0.001,0.10,2.0,0.068805,0.007260
7,274,0.1,0.010,0.01,1.0,0.068258,0.007929,0.1,0.010,5.00,...,0.50,1.0,0.068216,0.007922,0.1,0.010,5.00,1.0,0.068287,0.007936
8,276,0.1,0.001,5.00,1.0,0.112172,0.016299,0.1,0.001,1.00,...,0.01,1.0,0.112179,0.016304,0.1,0.001,1.00,1.0,0.112220,0.016313
9,328,0.1,0.001,0.01,5.0,0.067263,0.007506,0.1,0.001,0.01,...,0.01,5.0,0.067269,0.007508,0.1,0.001,0.01,5.0,0.067283,0.007511


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.1241,0.1242,0.1241,0.1242
1,0.1456,0.1456,0.1456,0.1457
2,1.6902,1.6902,1.6902,1.6902
3,0.1263,0.1263,0.1263,0.1263
4,0.0696,0.0695,0.0696,0.0695
5,0.0815,0.0814,0.0814,0.0814
6,0.0688,0.0688,0.0688,0.0688
7,0.0683,0.0682,0.0682,0.0683
8,0.1122,0.1122,0.1122,0.1122
9,0.0673,0.0673,0.0673,0.0673


Coefficient 1 : Polynomial_SVR 0.239 +/- 0.482
--------------------------------



In [13]:
i=2
for model in model_names:
    print(model)
    df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"}) 
    display(df)
    print("MAE among datasets:")
    display(round(df[["mae_"+str(i) for i in range(4)]], 4))
    print("Coefficient", i, ":", model, round(df["mae_3"].mean(), 3), "+/-", round(df["mae_3"].std(),3))
    print("--------------------------------\n")

lasso


,DRUG_ID,best_alpha0,mae_0,mse_0,intercept_0,sum_coef_0,best_alpha1,mae_1,mse_1,intercept_1,...,best_alpha2,mae_2,mse_2,intercept_2,sum_coef_2,best_alpha3,mae_3,mse_3,intercept_3,sum_coef_3
0,170,0.50,0.269558,0.074066,0.776455,0.000000,0.50,0.269558,0.074066,0.776455,...,0.50,0.269558,0.074066,0.776455,0.000000,0.50,0.269558,0.074066,0.776455,0.000000
1,173,0.10,0.222711,0.087835,1.143384,0.000000,0.10,0.222711,0.087835,1.143384,...,0.10,0.222711,0.087835,1.143384,0.000000,0.10,0.222711,0.087835,1.143384,0.000000
2,180,1.00,2.785129,30.956782,-0.176774,0.000000,1.00,2.785129,30.956782,-0.176774,...,1.00,2.785129,30.956782,-0.176774,0.000000,1.00,2.785129,30.956782,-0.176774,0.000000
3,200,0.01,0.052891,0.004577,0.989953,-5.350281,0.01,0.052891,0.004577,0.989953,...,0.01,0.052891,0.004577,0.989953,-5.350281,0.01,0.052891,0.004577,0.989953,-5.350281
4,219,0.50,0.183067,0.058246,1.030339,0.000000,0.50,0.183067,0.058246,1.030339,...,0.50,0.183067,0.058246,1.030339,0.000000,0.50,0.183067,0.058246,1.030339,0.000000
5,272,0.10,0.079124,0.013926,1.014484,0.000000,0.10,0.079124,0.013926,1.014484,...,0.10,0.079124,0.013926,1.014484,0.000000,0.10,0.079124,0.013926,1.014484,0.000000
6,273,0.10,0.083109,0.018349,1.069750,0.000000,0.10,0.083109,0.018349,1.069750,...,0.10,0.083109,0.018349,1.069750,0.000000,0.10,0.083109,0.018349,1.069750,0.000000
7,274,0.50,0.250585,0.079037,0.826107,0.000000,0.50,0.250585,0.079037,0.826107,...,0.50,0.250585,0.079037,0.826107,0.000000,0.50,0.250585,0.079037,0.826107,0.000000
8,276,0.01,0.039684,0.002329,1.024358,0.082853,0.01,0.039684,0.002329,1.024358,...,0.01,0.039684,0.002329,1.024358,0.082853,0.01,0.039684,0.002329,1.024358,0.082853
9,328,0.01,0.170109,0.042119,1.178561,-7.263998,0.01,0.170109,0.042119,1.178561,...,0.01,0.170109,0.042119,1.178561,-7.263998,0.01,0.170109,0.042119,1.178561,-7.263998


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.2696,0.2696,0.2696,0.2696
1,0.2227,0.2227,0.2227,0.2227
2,2.7851,2.7851,2.7851,2.7851
3,0.0529,0.0529,0.0529,0.0529
4,0.1831,0.1831,0.1831,0.1831
5,0.0791,0.0791,0.0791,0.0791
6,0.0831,0.0831,0.0831,0.0831
7,0.2506,0.2506,0.2506,0.2506
8,0.0397,0.0397,0.0397,0.0397
9,0.1701,0.1701,0.1701,0.1701


Coefficient 2 : lasso 0.391 +/- 0.798
--------------------------------

ridge


...

Coefficient 2 : RBF_SVR 0.272 +/- 0.554
--------------------------------

Polynomial_SVR


,DRUG_ID,best_C0,best_epsilon0,best_coef00,best_degree0,mae_0,mse_0,best_C1,best_epsilon1,best_coef01,...,best_coef02,best_degree2,mae_2,mse_2,best_C3,best_epsilon3,best_coef03,best_degree3,mae_3,mse_3
0,170,0.1,0.010,0.01,5.0,0.027704,0.001509,0.1,0.010,0.01,...,0.01,5.0,0.027698,0.001509,0.1,0.010,0.01,5.0,0.027605,0.001504
1,173,0.1,0.001,0.01,5.0,0.207895,0.085961,0.1,0.001,0.01,...,0.01,4.0,0.208902,0.086089,0.1,0.001,0.01,4.0,0.208857,0.086086
2,180,0.1,0.001,0.01,5.0,1.926118,34.087172,0.1,0.001,0.01,...,0.01,5.0,1.926126,34.087282,0.1,0.001,0.01,5.0,1.926129,34.087234
3,200,0.1,0.010,0.01,5.0,0.036227,0.002480,0.1,0.010,0.01,...,0.01,5.0,0.036200,0.002474,0.1,0.010,0.01,5.0,0.036170,0.002474
4,219,0.1,0.100,1.00,1.0,0.130541,0.027669,0.1,0.100,1.00,...,0.01,1.0,0.130536,0.027666,0.1,0.100,0.50,1.0,0.130511,0.027657
5,272,0.1,0.010,0.50,1.0,0.089104,0.012770,0.1,0.010,5.00,...,1.00,1.0,0.089104,0.012774,0.1,0.010,0.50,1.0,0.089080,0.012768
6,273,0.1,0.010,0.01,5.0,0.082181,0.018961,0.1,0.010,0.01,...,0.01,5.0,0.082175,0.018960,0.1,0.010,0.01,5.0,0.082171,0.018962
7,274,0.1,0.001,0.01,4.0,0.094870,0.019642,0.1,0.010,0.01,...,0.01,4.0,0.094883,0.019645,0.1,0.010,0.01,4.0,0.095657,0.019799
8,276,0.1,0.001,0.01,4.0,0.040921,0.002494,0.1,0.001,0.01,...,0.01,4.0,0.040960,0.002499,0.1,0.001,0.01,4.0,0.040914,0.002492
9,328,0.5,0.001,1.00,1.0,0.200357,0.058872,0.5,0.001,5.00,...,0.10,1.0,0.200341,0.058855,0.5,0.001,5.00,1.0,0.200266,0.058850


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.0277,0.0276,0.0277,0.0276
1,0.2079,0.2088,0.2089,0.2089
2,1.9261,1.9261,1.9261,1.9261
3,0.0362,0.0361,0.0362,0.0362
4,0.1305,0.1305,0.1305,0.1305
5,0.0891,0.0891,0.0891,0.0891
6,0.0822,0.0822,0.0822,0.0822
7,0.0949,0.0957,0.0949,0.0957
8,0.0409,0.0409,0.0410,0.0409
9,0.2004,0.2003,0.2003,0.2003


Coefficient 2 : Polynomial_SVR 0.273 +/- 0.552
--------------------------------



In [14]:
i=3
for model in model_names:
    print(model)
    df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"}) 
    display(df)
    print("MAE among datasets:")
    display(round(df[["mae_"+str(i) for i in range(4)]], 4))
    print("Coefficient", i, ":", model, round(df["mae_3"].mean(), 3), "+/-", round(df["mae_3"].std(),3))
    print("--------------------------------\n")

lasso


,DRUG_ID,best_alpha0,mae_0,mse_0,intercept_0,sum_coef_0,best_alpha1,mae_1,mse_1,intercept_1,...,best_alpha2,mae_2,mse_2,intercept_2,sum_coef_2,best_alpha3,mae_3,mse_3,intercept_3,sum_coef_3
0,170,5.0,13.108360,263.230022,-37.135827,0.000000,5.0,13.108360,263.230022,-37.135827,...,5.0,13.108360,263.230022,-37.135827,0.000000,5.0,13.108360,263.230022,-37.135827,0.000000
1,173,5.0,27.663468,2800.739979,-15.628001,0.000000,5.0,27.663468,2800.739979,-15.628001,...,5.0,27.663468,2800.739979,-15.628001,0.000000,5.0,27.663468,2800.739979,-15.628001,0.000000
2,180,0.1,7.939861,118.209679,-12.701594,-56.770755,0.1,7.939861,118.209679,-12.701594,...,0.1,7.939861,118.209679,-12.701594,-56.770755,0.1,7.939861,118.209679,-12.701594,-56.770755
3,200,0.5,25.433776,3694.015241,-10.752502,-9.045777,0.5,25.433776,3694.015241,-10.752502,...,0.5,25.433776,3694.015241,-10.752502,-9.045777,0.5,25.433776,3694.015241,-10.752502,-9.045777
4,219,10.0,14.827314,255.753441,-33.001120,0.000000,10.0,14.827314,255.753441,-33.001120,...,10.0,14.827314,255.753441,-33.001120,0.000000,10.0,14.827314,255.753441,-33.001120,0.000000
5,272,5.0,4.671322,35.847561,-13.664997,0.000000,5.0,4.671322,35.847561,-13.664997,...,5.0,4.671322,35.847561,-13.664997,0.000000,5.0,4.671322,35.847561,-13.664997,0.000000
6,273,1.0,7.633658,88.779735,-14.731020,-46.196125,1.0,7.633658,88.779735,-14.731020,...,1.0,7.633658,88.779735,-14.731020,-46.196125,1.0,7.633658,88.779735,-14.731020,-46.196125
7,274,1.0,4.497960,115.404796,-11.679018,0.000000,1.0,4.497960,115.404796,-11.679018,...,1.0,4.497960,115.404796,-11.679018,0.000000,1.0,4.497960,115.404796,-11.679018,0.000000
8,276,5.0,5.292148,40.507428,-15.856248,0.000000,5.0,5.292148,40.507428,-15.856248,...,5.0,5.292148,40.507428,-15.856248,0.000000,5.0,5.292148,40.507428,-15.856248,0.000000
9,328,5.0,4.209702,24.304098,-17.250994,0.000000,5.0,4.209702,24.304098,-17.250994,...,5.0,4.209702,24.304098,-17.250994,0.000000,5.0,4.209702,24.304098,-17.250994,0.000000


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,13.1084,13.1084,13.1084,13.1084
1,27.6635,27.6635,27.6635,27.6635
2,7.9399,7.9399,7.9399,7.9399
3,25.4338,25.4338,25.4338,25.4338
4,14.8273,14.8273,14.8273,14.8273
5,4.6713,4.6713,4.6713,4.6713
6,7.6337,7.6337,7.6337,7.6337
7,4.4980,4.4980,4.4980,4.4980
8,5.2921,5.2921,5.2921,5.2921
9,4.2097,4.2097,4.2097,4.2097


Coefficient 3 : lasso 11.132 +/- 8.374
--------------------------------

ridge


...

Coefficient 3 : RBF_SVR 9.509 +/- 8.793
--------------------------------

Polynomial_SVR


,DRUG_ID,best_C0,best_epsilon0,best_coef00,best_degree0,mae_0,mse_0,best_C1,best_epsilon1,best_coef01,...,best_coef02,best_degree2,mae_2,mse_2,best_C3,best_epsilon3,best_coef03,best_degree3,mae_3,mse_3
0,170,0.1,0.001,0.01,1.0,11.827324,265.841558,0.1,0.001,0.01,...,0.01,1.0,11.827371,265.843492,0.1,0.001,0.01,1.0,11.827719,265.857711
1,173,0.1,0.001,0.01,1.0,28.292027,2943.087173,0.1,0.001,0.01,...,0.01,1.0,28.292048,2943.088640,0.1,0.001,0.01,1.0,28.292207,2943.099420
2,180,0.1,0.001,0.01,1.0,6.499308,88.486812,0.1,0.001,0.01,...,0.01,1.0,6.499339,88.487635,0.1,0.001,0.01,1.0,6.499569,88.493694
3,200,0.1,0.001,0.01,1.0,25.565439,3714.728510,0.1,0.001,0.01,...,0.01,1.0,25.565459,3714.727681,0.1,0.001,0.01,1.0,25.565606,3714.721589
4,219,0.1,0.001,0.01,1.0,4.735037,35.147878,0.1,0.001,0.01,...,0.01,1.0,4.735051,35.148605,0.1,0.001,0.01,1.0,4.735149,35.153954
5,272,0.1,0.001,0.01,1.0,4.182994,31.908310,0.1,0.001,0.01,...,0.01,1.0,4.182991,31.908236,0.1,0.001,0.01,1.0,4.182972,31.907700
6,273,0.1,0.001,0.01,1.0,5.566210,60.222437,0.1,0.001,0.01,...,0.01,1.0,5.566213,60.222510,0.1,0.001,0.01,1.0,5.566234,60.223052
7,274,0.1,0.001,0.01,1.0,5.093117,124.022810,0.1,0.001,0.01,...,0.01,1.0,5.093111,124.023114,0.1,0.001,0.01,1.0,5.093067,124.025350
8,276,0.1,0.001,0.01,1.0,4.696755,42.262184,0.1,0.001,0.01,...,0.01,1.0,4.696775,42.261879,0.1,0.001,0.01,1.0,4.696920,42.259632
9,328,0.1,0.001,0.01,1.0,3.002530,14.604848,0.1,0.001,0.01,...,0.01,1.0,3.002548,14.604818,0.1,0.001,0.01,1.0,3.002621,14.604279


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,11.8273,11.8277,11.8274,11.8277
1,28.2920,28.2922,28.2920,28.2922
2,6.4993,6.4995,6.4993,6.4996
3,25.5654,25.5656,25.5655,25.5656
4,4.7350,4.7351,4.7351,4.7351
5,4.1830,4.1830,4.1830,4.1830
6,5.5662,5.5662,5.5662,5.5662
7,5.0931,5.0931,5.0931,5.0931
8,4.6968,4.6969,4.6968,4.6969
9,3.0025,3.0026,3.0025,3.0026


Coefficient 3 : Polynomial_SVR 9.589 +/- 8.882
--------------------------------



In [15]:
i=4
for model in model_names:
    print(model)
    df = pd.read_csv(_FOLDER+ model+"_coef"+str(i)+".csv").rename(columns={"Unnamed: 0": "DRUG_ID"}) 
    display(df)
    print("MAE among datasets:")
    display(round(df[["mae_"+str(i) for i in range(4)]], 4))
    print("Coefficient", i, ":", model, round(df["mae_3"].mean(), 3), "+/-", round(df["mae_3"].std(),3))
    print("--------------------------------\n")

lasso


,DRUG_ID,best_alpha0,mae_0,mse_0,intercept_0,sum_coef_0,best_alpha1,mae_1,mse_1,intercept_1,...,best_alpha2,mae_2,mse_2,intercept_2,sum_coef_2,best_alpha3,mae_3,mse_3,intercept_3,sum_coef_3
0,170,0.01,0.013103,0.000311,0.016271,0.000000,0.01,0.013103,0.000311,0.016271,...,0.01,0.013103,0.000311,0.016271,0.000000,0.01,0.013103,0.000311,0.016271,0.000000
1,173,0.01,0.177209,0.087881,0.069179,-5.189441,0.01,0.177209,0.087881,0.069179,...,0.01,0.177209,0.087881,0.069179,-5.189441,0.01,0.177209,0.087881,0.069179,-5.189441
2,180,0.01,0.133481,0.023014,0.211451,-0.026607,0.01,0.133481,0.023014,0.211451,...,0.01,0.133481,0.023014,0.211451,-0.026607,0.01,0.133481,0.023014,0.211451,-0.026607
3,200,0.10,0.030296,0.001756,0.018590,0.000000,0.10,0.030296,0.001756,0.018590,...,0.10,0.030296,0.001756,0.018590,0.000000,0.10,0.030296,0.001756,0.018590,0.000000
4,219,0.10,0.086981,0.010686,0.169612,0.000000,0.10,0.086981,0.010686,0.169612,...,0.10,0.086981,0.010686,0.169612,0.000000,0.10,0.086981,0.010686,0.169612,0.000000
5,272,0.10,0.065411,0.009340,0.037397,0.000000,0.10,0.065411,0.009340,0.037397,...,0.10,0.065411,0.009340,0.037397,0.000000,0.10,0.065411,0.009340,0.037397,0.000000
6,273,0.10,0.051902,0.006758,0.033392,0.000000,0.10,0.051902,0.006758,0.033392,...,0.10,0.051902,0.006758,0.033392,0.000000,0.10,0.051902,0.006758,0.033392,0.000000
7,274,0.01,0.053911,0.005869,0.068939,0.013154,0.01,0.053911,0.005869,0.068939,...,0.01,0.053911,0.005869,0.068939,0.013154,0.01,0.053911,0.005869,0.068939,0.013154
8,276,0.10,0.032060,0.001876,0.017873,0.000000,0.10,0.032060,0.001876,0.017873,...,0.10,0.032060,0.001876,0.017873,0.000000,0.10,0.032060,0.001876,0.017873,0.000000
9,328,0.10,0.102795,0.013503,0.188054,0.000000,0.10,0.102795,0.013503,0.188054,...,0.10,0.102795,0.013503,0.188054,0.000000,0.10,0.102795,0.013503,0.188054,0.000000


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.0131,0.0131,0.0131,0.0131
1,0.1772,0.1772,0.1772,0.1772
2,0.1335,0.1335,0.1335,0.1335
3,0.0303,0.0303,0.0303,0.0303
4,0.0870,0.0870,0.0870,0.0870
5,0.0654,0.0654,0.0654,0.0654
6,0.0519,0.0519,0.0519,0.0519
7,0.0539,0.0539,0.0539,0.0539
8,0.0321,0.0321,0.0321,0.0321
9,0.1028,0.1028,0.1028,0.1028


Coefficient 4 : lasso 0.078 +/- 0.05
--------------------------------

ridge


...

Coefficient 4 : RBF_SVR 0.075 +/- 0.047
--------------------------------

Polynomial_SVR


,DRUG_ID,best_C0,best_epsilon0,best_coef00,best_degree0,mae_0,mse_0,best_C1,best_epsilon1,best_coef01,...,best_coef02,best_degree2,mae_2,mse_2,best_C3,best_epsilon3,best_coef03,best_degree3,mae_3,mse_3
0,170,0.1,0.001,0.01,3.0,0.014212,0.000353,0.1,0.001,0.01,...,0.01,3.0,0.014194,0.000353,0.1,0.001,0.01,3.0,0.014184,0.000353
1,173,5.0,0.001,0.01,4.0,0.166018,0.084835,5.0,0.001,0.01,...,0.01,4.0,0.166021,0.084836,5.0,0.001,0.01,4.0,0.166107,0.084879
2,180,0.1,0.001,0.01,3.0,0.097669,0.015557,0.1,0.001,0.01,...,0.01,3.0,0.097650,0.015554,0.1,0.001,0.01,3.0,0.097576,0.015545
3,200,0.1,0.010,0.01,5.0,0.030625,0.001760,0.1,0.010,0.01,...,0.01,5.0,0.030624,0.001757,0.1,0.010,0.01,5.0,0.030600,0.001760
4,219,0.1,1.000,0.01,1.0,0.087949,0.010995,0.1,1.000,0.01,...,0.01,1.0,0.087949,0.010995,0.1,1.000,0.01,1.0,0.087949,0.010995
5,272,0.1,0.001,5.00,1.0,0.068051,0.010055,0.1,0.001,1.00,...,0.50,1.0,0.068089,0.010060,0.1,0.001,5.00,1.0,0.068029,0.010040
6,273,0.1,0.010,0.01,5.0,0.053505,0.007426,0.1,0.010,0.01,...,0.01,5.0,0.053525,0.007430,0.1,0.010,0.01,5.0,0.053565,0.007434
7,274,5.0,0.001,0.01,5.0,0.076645,0.011236,5.0,0.001,0.01,...,0.01,5.0,0.076689,0.011244,5.0,0.001,0.01,5.0,0.076636,0.011233
8,276,0.1,0.010,0.01,5.0,0.031910,0.001785,0.1,0.010,0.01,...,0.01,5.0,0.031907,0.001785,0.1,0.010,0.01,5.0,0.031903,0.001785
9,328,0.5,0.001,5.00,1.0,0.124842,0.020702,0.5,0.001,0.10,...,0.10,1.0,0.124805,0.020683,0.5,0.001,0.10,1.0,0.124601,0.020632


MAE among datasets:


,mae_0,mae_1,mae_2,mae_3
0,0.0142,0.0142,0.0142,0.0142
1,0.1660,0.1661,0.1660,0.1661
2,0.0977,0.0976,0.0977,0.0976
3,0.0306,0.0306,0.0306,0.0306
4,0.0879,0.0879,0.0879,0.0879
5,0.0681,0.0681,0.0681,0.0680
6,0.0535,0.0535,0.0535,0.0536
7,0.0766,0.0766,0.0767,0.0766
8,0.0319,0.0319,0.0319,0.0319
9,0.1248,0.1246,0.1248,0.1246


Coefficient 4 : Polynomial_SVR 0.078 +/- 0.045
--------------------------------

